# Test des fonctions d'accès aux données optiques.

_Version 1.0 [31/01/2023] Jérôme Lacaille_

In [1]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.rcParams["figure.figsize"] = (9,6) # Pour l'affichage d'images un peu plus grandes.

Vous trouverez ci-dessous un lien vers la doc de paramétrage de "matplotlib.

> [Customizing Matplotlib with style sheets and rcParams](https://matplotlib.org/stable/tutorials/introductory/customizing.html)

In [2]:
# Ce bout de code est pratique car il permet de recharger automatiquement un package quand vous le développez.
%reload_ext autoreload
%autoreload 2

## (Digression) Les fonction magiques.

Jupyter-Notebooks dispose de macros pratiques comme %matplotlib ou %autoreload. Consultez la doc.

In [ ]:
%magic

## 1. Gestion des zones géographiques.

Nous allons d'abord créer des fichiers de zones géographiques correspondant à des "rectangles" en latitude et longitudes.
Ce que l'on souhaite c'est pouvoir manipuler des dictionnaires avec deux composantes : un nom et les coordonnées du rectangle :

    {
        'name' : "Le nom de la zone",
        'bbox' : [lat_min, lon_min, lat_max, lon_max]
    }

Ensuite on va récupérer automatiquement ces rectangles depuis un site spécialisé et créer un nouvel objet Python capable de gérer ces listes.

## (Digression) Le Markdown.

Les cellules textes sont écrites en <mark>langage Markdown</mark> qui simplifie le HTML et inclue des fonctionnalités LaTeX.

> [Guide MarkDown](https://www.markdownguide.org/)

### 1.1 Etude d'une zone géographique.
En utilisant le site [geozone.io](https://geojson.io/) on peut créer facilement des GEOFILES qui contiennet des informations sur la latitude et la longitude de zones rectangulaires.

Un bouton vous permet de copier facilement un descriptif json (geojson en fait) et le copier directement dans une cellule comme un dictionnaire.

In [4]:
geofile = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {'name' : 'Chypre'},
      "geometry": {
        "coordinates": [
          [
            [
              23.554712962096346,
              35.607613541778534
            ],
            [
              23.554712962096346,
              34.89248636947745
            ],
            [
              26.302614086102693,
              34.89248636947745
            ],
            [
              26.302614086102693,
              35.607613541778534
            ],
            [
              23.554712962096346,
              35.607613541778534
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

Pour mieux gérer vos zones géographiques (geozones) je vous propose de rajouter un champ 'name' aux propriétés. Vous verrez que cela nous facilitera le travail par la suite.

          "properties": {'name' : 'Chypre'},

In [5]:
# On va avoir besoin de numpy pour gérer les coordinnées.
import numpy as np

On va maintenant récupérer le nom et les coordonnées.

In [6]:
name = geofile['features'][0]['properties']['name']
name

'Chypre'

In [7]:
G = np.array(geofile['features'][0]['geometry']['coordinates'][0])
G

array([[23.55471296, 35.60761354],
       [23.55471296, 34.89248637],
       [26.30261409, 34.89248637],
       [26.30261409, 35.60761354],
       [23.55471296, 35.60761354]])

On va maintenant chercher les coins de notre rectangle. Attention à ne pas intervertir longitude et latitude. Pour cela je vous propose de regarder un rectangle que vous caisssez bien, avec l'Ile de France par exemple, on ne peut pas se tromper.

In [8]:
lon0, lat0 = G.min(axis=0)
lat0, lon0

(34.89248636947745, 23.554712962096346)

In [9]:
lon1, lat1 = G.max(axis=0)
lat1, lon1

(35.607613541778534, 26.302614086102693)

Finalement vous pouvez créer votre zone comme un dictionnaire.

In [10]:
zone = dict(name=geofile['features'][0]['properties']['name'], bbox=[lat0, lon0, lat1, lon1])
zone

{'name': 'Chypre',
 'bbox': [34.89248636947745,
  23.554712962096346,
  35.607613541778534,
  26.302614086102693]}

### 1.2 Création d'une première fonction.

Le code que nous venons de créer peut s'automatiser. On va créer une fonction `getzone()` qui va prendre l'objet copié depuis geojson.io et renvoie la zone en question.

    def getzone(geofile):
        ...
        return zone

Cette fonction va être placée dans le module "geozone.py" mais vous pouvez déjà la mettre en oeuvre interactivement dans ce notebook.

In [11]:
def getzone(geolist):
    
    G0 = geolist['features'][0]

    # Récupération du nom de la zone.
    if 'name' in G0['properties']:
        name = G0['properties']['name']
    else:
        name = f"Geozone [{lat0:.1f},{lon0:.1f},{lat1:.1f},{lon1:.1f}]"

    G = np.array(G0['geometry']['coordinates'][0])
    lon0,lat0 = G.min(axis=0)
    lon1,lat1 = G.max(axis=0)
    zone = dict(
        name = name,
        bbox = [lat0, lon0, lat1, lon1]
    )
    return zone

Maintenant il faut tester si cette fonction marche bien.

In [12]:
getzone(geofile)

{'name': 'Chypre',
 'bbox': [34.89248636947745,
  23.554712962096346,
  35.607613541778534,
  26.302614086102693]}

Vous recopirez cette fonction au module "geozone" en rajoutant un paramètre `i` supplémentaire pour aller chercher chacune des zones de la liste fournie en argument.

    def getzone(geolist, i=0):
        ...

On force la valeur du paramètre `i` à 0, comme ça on peut toujours utiliser la fonction avec un seul argument.

<mark>N'oubliez pas d'importer le package.</mark>

In [13]:
# On importe la package satellite.
import satellite as sat

In [14]:
# On teste la fonction programmée dans le package.
sat.geozone.getzone(geofile)

{'name': 'Chypre',
 'bbox': [34.89248636947745,
  23.554712962096346,
  35.607613541778534,
  26.302614086102693]}

Pour que la fonction soit plus facile d'accès on rajoute `getzone` à l'import de "geozone" dans "__init__.py".

    from .geozone import getzone, EARTHDIR

In [15]:
# On teste notre facilité d'import.
sat.getzone(geofile)

{'name': 'Chypre',
 'bbox': [34.89248636947745,
  23.554712962096346,
  35.607613541778534,
  26.302614086102693]}

### 1.3 Gestion des erreurs.

Quand on écrit un package, il faut faire attention à ne pas risquer de produire des erreurs incontrôlées. On va donc ajouter quelques éléments de gestion d'erreur à notre fonction.

Python gère les erreurs à l'aide d'exceptions. Plusieurs types d'exceptions sont disponibles (voir la documentation de Python sur les <mark>Exceptions</mark>)

> [Built-in Exceptions](https://docs.python.org/3/library/exceptions.html#base-classes)

Il est possible de créer des exceptions en exécutant par exemple un appel à l'instruction `assert`, mais cela génère une `AssertionError` qui reste assez générique. On peut aussi vouloir créer sa propre execption et dans ce cas on va dériver une classe exception et fabriquer la notre.

Dans notre cas on va créer une classe dérivée de `ValueError` qui est une exception assez standard quand on a un problème du aux données d'entrée d'une fonction.

Vous rajouterez la création d'une classe spécifique `GeofileException` au module geozone qui dérive de `ValueError`. Comme cette Exception se comporte exactement comme `ValueError` il n'y a rien d'autre à faire qu'à la déclarer. Quand elle sera lancée on verra tout de suite si l'erreur vient de l'objet.

Pour exécuter une exception on utilise la commande `raise`.

In [16]:
raise sat.geozone.GeofileException("Un essai.")

GeofileException: Un essai.

Nous utilisezons cette exception pour tester l'existance des clés 'properties', 'geometry' et 'coordinates' dans le paramètre envoyé en entrée.

Ci-dessous j'ai remplacé la clé 'geometry' par 'geometrie', et je teste si mon code réagit bien.

In [17]:
badgeofile = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {'name' : 'Chypre'},
      "geometrie": {
        "coordinates": [
          [
            [
              23.554712962096346,
              35.607613541778534
            ],
            [
              23.554712962096346,
              34.89248636947745
            ],
            [
              26.302614086102693,
              34.89248636947745
            ],
            [
              26.302614086102693,
              35.607613541778534
            ],
            [
              23.554712962096346,
              35.607613541778534
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [18]:
sat.getzone(badgeofile)

GeofileException: No geometry in geolist[0].

C'est plus clair ! On voit tout de suite que c'est un problème d'entrée.

Si on veut tester qu'un argument entier est bien dans les bornes attendues, ce que l'on fait en général c'es appeler directement `assert()`.
C'est ce que l'on va faire pour vérifier si l'entier passé est bien entre 0 et la longeur de la liste de zones géographiques donnés en entrée.

In [19]:
sat.getzone(geofile,-1)

AssertionError: Le numéro de zone doit être positif.

In [20]:
sat.getzone(geofile,3)

AssertionError: La liste ne contient que 1 zone.

**Astuce** : J'ai utilisé l'opérateur ternaire pour m'assure que "zone" sera au singulier si le nombres de zones est inféreiyer ou égal à 1 et au pluriel sinon.

    n = len(geolist['features'])
    sz = "zones" if n>1 else "zone"

**Et si on avait oublié le nom ?**
Vous vous souvenez que j'avais proposé de rajouter le nom de votre zone dans le champ "properties". On ne va toute de même pas créer une erreur à cause de cela, je vous propose dans le cas d'un nom absent de créer un texte à partir des coordonnées.


In [21]:
notnamedgeofile = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              23.554712962096346,
              35.607613541778534
            ],
            [
              23.554712962096346,
              34.89248636947745
            ],
            [
              26.302614086102693,
              34.89248636947745
            ],
            [
              26.302614086102693,
              35.607613541778534
            ],
            [
              23.554712962096346,
              35.607613541778534
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

On essaue de se limiter à une seule valeur après la virgule.

In [22]:
sat.getzone(notnamedgeofile)

{'name': 'Geozone [34.9,23.6,35.6,26.3]',
 'bbox': [34.89248636947745,
  23.554712962096346,
  35.607613541778534,
  26.302614086102693]}

### 1.4 Gestion d'une liste de zones géographiques.

Maintenant nous allons profiter d'une fonction sympatique de [geozone.io](https://geojson.io/) qui nous permet de sélectionner en une seule fois plusieurs zones. Une fois que c'est fait enregistrez le fichier. Par défaut, il s'appelle "map.geojson".

Je vous propose de placer ce fichier dans le sous-répertoire pointé par EARTHDIR.

Créez cette fois une nouvelle fonction `zonelist()` qui renvoie la liste de toutes les zones du fichier. Par défaut, cette fonction av chercher le fichier dans EARTHDIR, et prend "map.geojson" comme valeur par défaut.

    def zonelist(geomap  "map.geojson")

Pour commencer il faut convertir le fichier "map.geojson" en un dictionnaire. Le plus simple est d'utiliser le module json de Python.

In [23]:
import os,json

In [24]:
geomapfile = os.path.join(sat.EARTHDIR,'map.geojson')
geomapfile

'/Users/holie/wrk0/satellite/data/zones/_Earth/map.geojson'

Je préfère toujours utiliser l'instruction `with` pour gerer proprement les ressoures.

In [25]:
with open(geomapfile) as g:
    geolist = json.load(g)
geolist

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Ile de France'},
   'geometry': {'coordinates': [[[2.0085203859495095, 48.99193420772434],
      [2.0085203859495095, 48.7535883521073],
      [2.6788390690815334, 48.7535883521073],
      [2.6788390690815334, 48.99193420772434],
      [2.0085203859495095, 48.99193420772434]]],
    'type': 'Polygon'}},
  {'type': 'Feature',
   'properties': {'name': 'Barcelone'},
   'geometry': {'coordinates': [[[1.6649881237897546, 41.63394152808198],
      [1.6649881237897546, 41.22007856049126],
      [2.4690345841926273, 41.22007856049126],
      [2.4690345841926273, 41.63394152808198],
      [1.6649881237897546, 41.63394152808198]]],
    'type': 'Polygon'}},
  {'type': 'Feature',
   'properties': {'name': 'Chypre'},
   'geometry': {'coordinates': [[[23.397028867484522, 35.631415093017225],
      [23.397028867484522, 34.85709260445668],
      [26.44578481885614, 34.85709260445668],
      [26.44578481885614, 3

Voila ! Vous avez ce qu'il faut pour rajouter `zonelist()` à "geozone". N'oubliez pas la gestion des erreurs.

In [26]:
Z = sat.zonelist()
Z

[{'name': 'Ile de France',
  'bbox': [48.7535883521073,
   2.0085203859495095,
   48.99193420772434,
   2.6788390690815334]},
 {'name': 'Barcelone',
  'bbox': [41.22007856049126,
   1.6649881237897546,
   41.63394152808198,
   2.4690345841926273]},
 {'name': 'Chypre',
  'bbox': [34.85709260445668,
   23.397028867484522,
   35.631415093017225,
   26.44578481885614]}]

## 2. Construction d'une classe

Maintenant nous allons regrouper tout ce que nous avons appris pour faire une classe `GeoZone` initialisée avec un gichier "map.geojson".

Cette classe doit avoir les fonctions suivante.

* S'initialiser par "map.gojson".

        gz = Geozone()

* Renvoyer sa longeur
    
        len(gz)

* Iterer sur les zones
    
        for z in gz:
            ...

* Accéder à une zone par indexation.
    
        z = gz[2]
